# Modeling

In [94]:
import pandas as pd
df = pd.read_csv("../Data/HanRiver_FeatureEngineering.csv")

In [95]:
df.drop(columns='Unnamed: 0', axis=1, inplace=True)

In [112]:
df.columns

Index(['지구별', '주차장명', '주차대수', '이용시간', '월', '일', '휴일여부', '평균_주차시간(분)',
       '평균기온(°C)', '주차대수_평균', '총_주차가능_대수', '일반이용자_합계_일별', '일반이용자_아침_일별',
       '일반이용자_낮_일별', '일반이용자_저녁_일별', '주차 가능 확률', '주차대수_아침', '주차대수_낮', '주차대수_저녁',
       '점유율_아침', '점유율_낮', '점유율_저녁', '혼잡도_아침', '혼잡도_낮', '혼잡도_저녁'],
      dtype='object')

> 모델 성능을 정확히 평가하고, 일반화 능력을 평가하기 위해 훈련, 검증, 테스트 세트로 데이터를 나눠준다.

In [97]:
from sklearn.model_selection import train_test_split

# 타겟과 피처 설정
targets = ['혼잡도_아침', '혼잡도_낮', '혼잡도_저녁']
features = df.drop(columns=targets).columns

# 결과 저장을 위한 DataFrame 생성
result_df = pd.DataFrame(columns=['Target', 'Model', 'Set', 'MSE', 'R^2'])

# 8:2 비율로 훈련 세트와 테스트 세트 나누기
def split_data(df, target):
    X = df[features]
    y = df[target]

    # 첫 번째 분할: 전체 데이터를 훈련 세트와 테스트 세트로 분리
    train_input, test_input, train_target, test_target = train_test_split(
        X, y, test_size=0.2, random_state=42
    )
    
    # 두 번째 분할: 훈련 세트를 다시 훈련 세트와 검증 세트로 분리
    sub_input, val_input, sub_target, val_target = train_test_split(
        train_input, train_target, test_size=0.2, random_state=42
    )

    return sub_input, val_input, test_input, sub_target, val_target, test_target

### 선형회귀

In [108]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

for target in targets:
    print(f"\n===== {target} - Linear Regression =====")
    
    # 데이터 분할
    sub_input, val_input, test_input, sub_target, val_target, test_target = split_data(df, target)
    
    # 스케일링 (정규화)
    scaler = StandardScaler()
    sub_input_scaled = scaler.fit_transform(sub_input)
    val_input_scaled = scaler.transform(val_input)
    test_input_scaled = scaler.transform(test_input)
    
    # 모델 학습 및 평가
    model = LinearRegression()
    model.fit(sub_input_scaled, sub_target)
    
    # 훈련 세트 평가
    train_pred = model.predict(sub_input_scaled)
    train_mse = mean_squared_error(sub_target, train_pred)
    train_r2 = r2_score(sub_target, train_pred)
    train_score = model.score(sub_input_scaled, sub_target)
    print(f"훈련 세트 MSE: {train_mse:.3f}, R^2: {train_r2:.3f}, Score: {train_score:.3f}")
    
    # 검증 세트 평가
    val_pred = model.predict(val_input_scaled)
    val_mse = mean_squared_error(val_target, val_pred)
    val_r2 = r2_score(val_target, val_pred)
    val_score = model.score(val_input_scaled, val_target)
    print(f"검증 세트 MSE: {val_mse:.3f}, R^2: {val_r2:.3f}, Score: {val_score:.3f}")
    
    # 테스트 세트 평가
    test_pred = model.predict(test_input_scaled)
    test_mse = mean_squared_error(test_target, test_pred)
    test_r2 = r2_score(test_target, test_pred)
    test_score = model.score(test_input_scaled, test_target)
    print(f"테스트 세트 MSE: {test_mse:.3f}, R^2: {test_r2:.3f}, Score: {test_score:.3f}")

    # 모델의 일반화 성능
    print(f"일반화 성능 (Train Score): {train_score:.3f}")
    print(f"테스트 성능 (Test Score): {test_score:.3f}")


===== 혼잡도_아침 - Linear Regression =====
훈련 세트 MSE: 0.700, R^2: 0.218, Score: 0.218
검증 세트 MSE: 0.698, R^2: 0.219, Score: 0.219
테스트 세트 MSE: 0.694, R^2: 0.213, Score: 0.213
일반화 성능 (Train Score): 0.218
테스트 성능 (Test Score): 0.213

===== 혼잡도_낮 - Linear Regression =====
훈련 세트 MSE: 1.440, R^2: 0.220, Score: 0.220
검증 세트 MSE: 1.445, R^2: 0.213, Score: 0.213
테스트 세트 MSE: 1.468, R^2: 0.210, Score: 0.210
일반화 성능 (Train Score): 0.220
테스트 성능 (Test Score): 0.210

===== 혼잡도_저녁 - Linear Regression =====
훈련 세트 MSE: 1.354, R^2: 0.273, Score: 0.273
검증 세트 MSE: 1.378, R^2: 0.259, Score: 0.259
테스트 세트 MSE: 1.359, R^2: 0.274, Score: 0.274
일반화 성능 (Train Score): 0.273
테스트 성능 (Test Score): 0.274


### 랜덤 포레스트

In [109]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

for target in targets:
    print(f"\n===== {target} - Random Forest =====")
    
    # 데이터 분할
    sub_input, val_input, test_input, sub_target, val_target, test_target = split_data(df, target)
    
    # 스케일링 (정규화)
    scaler = StandardScaler()
    sub_input_scaled = scaler.fit_transform(sub_input)
    val_input_scaled = scaler.transform(val_input)
    test_input_scaled = scaler.transform(test_input)
    
    # 모델 학습 및 평가
    model = RandomForestRegressor(random_state=42)
    model.fit(sub_input_scaled, sub_target)
    
    # 훈련 세트 평가
    train_pred = model.predict(sub_input_scaled)
    train_mse = mean_squared_error(sub_target, train_pred)
    train_r2 = r2_score(sub_target, train_pred)
    train_score = model.score(sub_input_scaled, sub_target)
    print(f"훈련 세트 MSE: {train_mse:.3f}, R^2: {train_r2:.3f}, Score: {train_score:.3f}")
    
    # 검증 세트 평가
    val_pred = model.predict(val_input_scaled)
    val_mse = mean_squared_error(val_target, val_pred)
    val_r2 = r2_score(val_target, val_pred)
    val_score = model.score(val_input_scaled, val_target)
    print(f"검증 세트 MSE: {val_mse:.3f}, R^2: {val_r2:.3f}, Score: {val_score:.3f}")
    
    # 테스트 세트 평가
    test_pred = model.predict(test_input_scaled)
    test_mse = mean_squared_error(test_target, test_pred)
    test_r2 = r2_score(test_target, test_pred)
    test_score = model.score(test_input_scaled, test_target)
    print(f"테스트 세트 MSE: {test_mse:.3f}, R^2: {test_r2:.3f}, Score: {test_score:.3f}")

    # 모델의 일반화 성능
    print(f"일반화 성능 (Train Score): {train_score:.3f}")
    print(f"테스트 성능 (Test Score): {test_score:.3f}")



===== 혼잡도_아침 - Random Forest =====
훈련 세트 MSE: 0.000, R^2: 1.000, Score: 1.000
검증 세트 MSE: 0.000, R^2: 1.000, Score: 1.000
테스트 세트 MSE: 0.001, R^2: 0.999, Score: 0.999
일반화 성능 (Train Score): 1.000
테스트 성능 (Test Score): 0.999

===== 혼잡도_낮 - Random Forest =====
훈련 세트 MSE: 0.000, R^2: 1.000, Score: 1.000
검증 세트 MSE: 0.000, R^2: 1.000, Score: 1.000
테스트 세트 MSE: 0.001, R^2: 0.999, Score: 0.999
일반화 성능 (Train Score): 1.000
테스트 성능 (Test Score): 0.999

===== 혼잡도_저녁 - Random Forest =====
훈련 세트 MSE: 0.000, R^2: 1.000, Score: 1.000
검증 세트 MSE: 0.000, R^2: 1.000, Score: 1.000
테스트 세트 MSE: 0.000, R^2: 1.000, Score: 1.000
일반화 성능 (Train Score): 1.000
테스트 성능 (Test Score): 1.000


### 그레디언트 부스팅

In [101]:
from sklearn.ensemble import GradientBoostingRegressor

for target in targets:
    print(f"\n===== {target} - Gradient Boosting =====")
    
    # 데이터 분할
    sub_input, val_input, test_input, sub_target, val_target, test_target = split_data(df, target)
    
    # 스케일링 (정규화)
    scaler = StandardScaler()
    sub_input_scaled = scaler.fit_transform(sub_input)
    val_input_scaled = scaler.transform(val_input)
    test_input_scaled = scaler.transform(test_input)
    
    # 모델 학습 및 평가
    model = GradientBoostingRegressor(random_state=42)
    model.fit(sub_input_scaled, sub_target)
    
    # 훈련 세트 평가
    save_results(target, 'Gradient Boosting', 'Train', sub_target, model.predict(sub_input_scaled))
    
    # 검증 세트 평가
    save_results(target, 'Gradient Boosting', 'Validation', val_target, model.predict(val_input_scaled))
    
    # 테스트 세트 평가
    save_results(target, 'Gradient Boosting', 'Test', test_target, model.predict(test_input_scaled))



===== 혼잡도_아침 - Gradient Boosting =====

===== 혼잡도_낮 - Gradient Boosting =====

===== 혼잡도_저녁 - Gradient Boosting =====


### KNN

In [102]:
from sklearn.neighbors import KNeighborsRegressor

for target in targets:
    print(f"\n===== {target} - K-Nearest Neighbors =====")
    
    # 데이터 분할
    sub_input, val_input, test_input, sub_target, val_target, test_target = split_data(df, target)
    
    # 스케일링 (정규화)
    scaler = StandardScaler()
    sub_input_scaled = scaler.fit_transform(sub_input)
    val_input_scaled = scaler.transform(val_input)
    test_input_scaled = scaler.transform(test_input)
    
    # 모델 학습 및 평가
    model = KNeighborsRegressor()
    model.fit(sub_input_scaled, sub_target)
    
    # 훈련 세트 평가
    save_results(target, 'K-Nearest Neighbors', 'Train', sub_target, model.predict(sub_input_scaled))
    
    # 검증 세트 평가
    save_results(target, 'K-Nearest Neighbors', 'Validation', val_target, model.predict(val_input_scaled))
    
    # 테스트 세트 평가
    save_results(target, 'K-Nearest Neighbors', 'Test', test_target, model.predict(test_input_scaled))



===== 혼잡도_아침 - K-Nearest Neighbors =====

===== 혼잡도_낮 - K-Nearest Neighbors =====

===== 혼잡도_저녁 - K-Nearest Neighbors =====


### SVM

In [110]:
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score

for target in targets:
    print(f"\n===== {target} - Support Vector Regressor =====")
    
    # 데이터 분할
    sub_input, val_input, test_input, sub_target, val_target, test_target = split_data(df, target)
    
    # 스케일링 (정규화)
    scaler = StandardScaler()
    sub_input_scaled = scaler.fit_transform(sub_input)
    val_input_scaled = scaler.transform(val_input)
    test_input_scaled = scaler.transform(test_input)
    
    # 모델 학습 및 평가
    model = SVR()
    model.fit(sub_input_scaled, sub_target)
    
    # 훈련 세트 평가
    train_pred = model.predict(sub_input_scaled)
    train_mse = mean_squared_error(sub_target, train_pred)
    train_r2 = r2_score(sub_target, train_pred)
    train_score = model.score(sub_input_scaled, sub_target)
    print(f"훈련 세트 MSE: {train_mse:.3f}, R^2: {train_r2:.3f}, Score: {train_score:.3f}")
    
    # 검증 세트 평가
    val_pred = model.predict(val_input_scaled)
    val_mse = mean_squared_error(val_target, val_pred)
    val_r2 = r2_score(val_target, val_pred)
    val_score = model.score(val_input_scaled, val_target)
    print(f"검증 세트 MSE: {val_mse:.3f}, R^2: {val_r2:.3f}, Score: {val_score:.3f}")
    
    # 테스트 세트 평가
    test_pred = model.predict(test_input_scaled)
    test_mse = mean_squared_error(test_target, test_pred)
    test_r2 = r2_score(test_target, test_pred)
    test_score = model.score(test_input_scaled, test_target)
    print(f"테스트 세트 MSE: {test_mse:.3f}, R^2: {test_r2:.3f}, Score: {test_score:.3f}")

    # 모델의 일반화 성능
    print(f"일반화 성능 (Train Score): {train_score:.3f}")
    print(f"테스트 성능 (Test Score): {test_score:.3f}")



===== 혼잡도_아침 - Support Vector Regressor =====
훈련 세트 MSE: 0.313, R^2: 0.651, Score: 0.651
검증 세트 MSE: 0.345, R^2: 0.614, Score: 0.614
테스트 세트 MSE: 0.344, R^2: 0.610, Score: 0.610
일반화 성능 (Train Score): 0.651
테스트 성능 (Test Score): 0.610

===== 혼잡도_낮 - Support Vector Regressor =====
훈련 세트 MSE: 0.489, R^2: 0.735, Score: 0.735
검증 세트 MSE: 0.540, R^2: 0.706, Score: 0.706
테스트 세트 MSE: 0.574, R^2: 0.691, Score: 0.691
일반화 성능 (Train Score): 0.735
테스트 성능 (Test Score): 0.691

===== 혼잡도_저녁 - Support Vector Regressor =====
훈련 세트 MSE: 0.473, R^2: 0.746, Score: 0.746
검증 세트 MSE: 0.527, R^2: 0.717, Score: 0.717
테스트 세트 MSE: 0.502, R^2: 0.732, Score: 0.732
일반화 성능 (Train Score): 0.746
테스트 성능 (Test Score): 0.732


#### 출력

In [106]:
# results_df 리스트를 DataFrame으로 변환하여 리포트 출력
results_df = pd.DataFrame(results_df)
print("모델 성능 평가 리포트:")
print(results_df)

모델 성능 평가 리포트:
    Target                     Model         Set           MSE       R^2
0   혼잡도_아침         Linear Regression       Train  7.003772e-01  0.218317
1   혼잡도_아침         Linear Regression  Validation  6.975959e-01  0.219121
2   혼잡도_아침         Linear Regression        Test  6.939467e-01  0.213338
3    혼잡도_낮         Linear Regression       Train  1.440314e+00  0.220427
4    혼잡도_낮         Linear Regression  Validation  1.444759e+00  0.213319
5    혼잡도_낮         Linear Regression        Test  1.467738e+00  0.210123
6   혼잡도_저녁         Linear Regression       Train  1.353510e+00  0.272607
7   혼잡도_저녁         Linear Regression  Validation  1.378110e+00  0.259129
8   혼잡도_저녁         Linear Regression        Test  1.358786e+00  0.273631
9   혼잡도_아침             Random Forest       Train  7.307482e-06  0.999992
10  혼잡도_아침             Random Forest  Validation  1.397571e-04  0.999844
11  혼잡도_아침             Random Forest        Test  7.238573e-04  0.999179
12   혼잡도_낮             Random Forest 